# Imports

The code above imports various Python modules and libraries for data processing, visualization, and analysis. Below is a description of each module and library imported:

- **os** - This module provides a portable way of using operating system dependent functionality. It is used to create subdirectories within the base folder path.
- **spacy** - This module is used to perform natural language processing (NLP) tasks. It is used to extract the location from the 'Description' property value.
- **folium** - This module is used to create interactive maps. It is used to create the map of the images' locations.
- **sqlite3** - This module is used to create a connection to a SQLite database. It is used to retrieve metadata from the database.
- **datetime** - This module is used to convert the 'DateTime' property value to a datetime object. It is used to create the map of the images' locations.
- **squarify** - This module is used to create treemap.
- **webcolors** - This module is used to convert color names to RGB values. It is used to get the most common colors in the images.
- **collections** - This module is used to create a Counter object. It is used to count the number of images with a specific property value.
- **numpy** - This module is used to perform various mathematical operations. It is used to calculate the mean and standard deviation of the images' dimensions.
- **pandas** - This module is used to create a DataFrame object. It is used to create a DataFrame of the images' metadata.
- **tqdm** - This module is used to create a progress bar. It is used to show the progress of many of the functions.
- **mysql.connector** - This module is used to create a connection to a MariaDB database. It is used to retrieve metadata from the database.
- **load_dotenv** - This function is used to load the environment variables from the .env file. It is used to set the SQL variables.
- **matplotlib.pyplot** - This module is used to create various types of plots. It is used to create all the plots in this notebook.
- **geopy.geocoders** - This module is used to geocode the location of the images. It is used to create the map of the images' locations.
- **scipy.spatial.distance** - This module is used to calculate the distance between two points. It is used to create the dendrogram.
- **scipy.cluster.hierarchy** - This module is used to create a dendrogram. It is used to create the dendrogram.
- **concurrent.futures** - This module is used to create a ThreadPoolExecutor object. It is used to create the map of the images' locations.

In [ ]:
!pip install spacy folium mysql-connector-python python-dotenv squarify webcolors numpy pandas tqdm mysql-connector-python python-dotenv matplotlib geopy scipy

In [ ]:
import os
import spacy
import folium
import sqlite3
import datetime
import squarify
import webcolors
import collections
import numpy as np
import pandas as pd
import mysql.connector
import concurrent.futures
import matplotlib.pyplot as plt

from tqdm import tqdm
from dotenv import load_dotenv
from collections import Counter
from geopy.geocoders import Nominatim
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram, linkage


# Settings base variables and paths

The code above sets the basic variables and paths for the project, but also loads the environment variables from the .env file.

In [ ]:
load_dotenv()

# Set the base folder path for the project
output_path = "../output"
images_path = os.path.join(output_path, "images")
metadata_path = os.path.join(output_path, "metadata")
config_path = os.path.join(output_path, "config")

list_of_paths = [output_path, images_path, metadata_path, config_path]

# Set SQL variables
sql_host = os.getenv("SQL_HOST")
sql_user = os.getenv("SQL_USER")
sql_password = os.getenv("SQL_PASSWORD")
sql_database = os.getenv("SQL_DATABASE")

# Get the metadata from db and clean it

The code above retrieves the metadata from the database and cleans it. It creates a dataframe with all the metadata.

### Get the metadata from the database

The function "get_metadata_from_sqlite_db" is used to retrieve the metadata from the SQLite database. This function is not used in the notebook anymore since we get our metadata from a remote mariadb database. It was useful to get the metadata from the local database when we were still developing the project.

In [ ]:
def get_metadata_from_sqlite_db(db_name='metadata.db'):
    """
    !! UNUSED !!
    Get the metadata from the sqlite database

    :param db_name: The name of the database
    :return: A dictionary with the metadata
    """
    # Open a connection to the database
    conn = sqlite3.connect(os.path.join(metadata_path, db_name))
    # Create a cursor
    c = conn.cursor()

    # Retrieve the metadata
    c.execute("""
        SELECT filename, GROUP_CONCAT(key || '\t' || value, '\n') AS metadata
        FROM metadata
        GROUP BY filename;
    """)
    metadata = c.fetchall()

    # Close the connection
    conn.close()

    # Convert the metadata to a DataFrame
    result = {}
    for image in tqdm(metadata, desc="Get metadata from database"):
        try:
            result[image[0]] = {}
            props = image[1].split('\n')
            for prop in props:
                if prop:
                    k, value = prop.split('\t')
                    result[image[0]][k] = value
        except Exception as e:
            print(e, image)

    return result

The function "get_metadata_from_mariadb_db" is used to retrieve the metadata from the remote MariaDB database. It returns a dictionary with the metadata.

In [ ]:
def get_metadata_from_mariadb_db(db_name='bigdata', user='root', password='', host='localhost', port='3306'):
    """
    Get the metadata from the MariaDB database

    :param db_name: The name of the database
    :param user: The username to connect to the database
    :param password: The password to connect to the database
    :param host: The hostname or IP address of the database server
    :param port: The port number to connect to the database server
    :return: A dictionary with the metadata
    """
    print("Connecting to database...")

    # Open a connection to the database
    conn = mysql.connector.connect(
        user=user,
        password=password,
        host=host,
        port=port,
        database=db_name
    )
    # Create a cursor
    c = conn.cursor()

    print("Retrieving metadata from database...")

    # Retrieve the metadata
    c.execute("""
        SELECT filename, GROUP_CONCAT(CONCAT(mkey, '\t', mvalue) SEPARATOR '\n') AS metadata
        FROM metadata
        GROUP BY filename;
    """)
    metadata = c.fetchall()

    # Close the connection
    conn.close()

    # Convert the metadata to a dictionary
    result = {}
    for image in tqdm(metadata, desc="Get metadata from database"):
        try:
            result[image[0]] = {}
            props = image[1].split('\n')
            for prop in props:
                if prop:
                    k, value = prop.split('\t')
                    result[image[0]][k] = value
        except Exception as e:
            print(e, image)

    return result

### Clean the metadata

The function "dms_to_decimal" is used to convert the GPS coordinates from DMS (degrees, minutes, seconds) to DD (decimal degrees). It is used to clean the GPS coordinates.

In [ ]:
def dms_to_decimal(degrees, minutes, seconds):
    """
    Convert DMS (degrees, minutes, seconds) coordinates to DD (decimal degrees)
    :param degrees: degrees
    :param minutes: minutes
    :param seconds:  seconds
    :return: decimal coordinates
    """
    decimal_degrees = abs(degrees) + (minutes / 60) + (seconds / 3600)

    if degrees < 0:
        decimal_degrees = -decimal_degrees

    return decimal_degrees

The function "clean_gps_infos" is used to clean the GPS coordinates. It checks if the GPS coordinates are in DMS or DD format and converts them to DD if needed. It also checks if the GPS coordinates are valid or not. It returns a dictionary with the cleaned metadata.

In [ ]:
def clean_gps_infos(metadata_to_cln):
    """
    Clean the GPS infos

    :param metadata_to_cln: The metadata to clean
    :return: A dictionary with the cleaned metadata
    """

    cpt_valid, cpt_invalid, cpt_converted = 0, 0, 0
    for file in tqdm(metadata_to_cln, desc="Clean GPS values"):
        file_meta = metadata_to_cln[file]

        if 'Latitude' in file_meta:
            has_dms_values = file_meta['LatitudeDegrees'] != '0.000000' or file_meta['LongitudeDegrees'] != '0.000000'
            has_decimal_values = file_meta['Latitude'] != '0.000000' or file_meta['Longitude'] != '0.000000'

            if has_dms_values or has_decimal_values:
                try:
                    should_convert = '.' not in file_meta['Latitude'] and has_dms_values
                except:
                    continue

                if should_convert:
                    # calculate the decimal coordinates from the degrees coordinates
                    latitude = dms_to_decimal(
                        float(file_meta['LatitudeDegrees']),
                        float(file_meta['LatitudeMinutes']),
                        float(file_meta['LatitudeSeconds']))

                    longitude = dms_to_decimal(
                        float(file_meta['LongitudeDegrees']),
                        float(file_meta['LongitudeMinutes']),
                        float(file_meta['LongitudeSeconds']))

                    cpt_converted += 1
                else:
                    # convert the coordinates to float
                    latitude = float(file_meta['Latitude'])
                    longitude = float(file_meta['Longitude'])

                # update the metadata with the calculated latitude and longitude
                metadata_to_cln[file]['Latitude'] = latitude
                metadata_to_cln[file]['Longitude'] = longitude
                cpt_valid += 1

            else:
                metadata_to_cln[file]['Latitude'] = None
                metadata_to_cln[file]['Longitude'] = None
                metadata_to_cln[file]['Altitude'] = None
                cpt_invalid += 1

    print("GPS values : \n",
          "Valid : ", cpt_valid,
          "\nInvalid : ", cpt_invalid,
          "\nConverted : ", cpt_converted,
          )

    return metadata_to_cln

The function "clean_metadata" is used to clean the metadata. It removes useless characters from the 'Make' property values and format the 'DateTime' property values. It also clean the tags property values by converting them to list instead of string, and calls the function "clean_gps_infos" to clean the GPS coordinates. It returns a dictionary with the cleaned metadata.

In [ ]:
def clean_metadata(metadata_to_clean):
    """
    Clean the metadata
    Remove special characters from the 'Make' property values
    Remove the 'T' and '-' characters from the 'DateTime' property values

    :param metadata_to_clean: The metadata to clean
    :return: A dictionary with the cleaned metadata
    """
    cln_meta = metadata_to_clean.copy()

    # Clean 'Make' property values
    try:
        for file in tqdm(cln_meta, desc="Clean 'Make' property values"):
            if 'Make' in cln_meta[file]:
                cln_meta[file]['Make'] = ''.join(filter(str.isalpha, cln_meta[file]['Make'])).replace('CORPORATION','').replace('CORP', '').replace('COMPANY', '').replace('LTD', '').replace('IMAGING', '')
    except Exception as e:
        print(e)

    # Clean 'DateTime' property values
    cpt, cpt_error = 0, 0
    date_error = []
    try:

        for file in tqdm(cln_meta, desc="Clean 'DateTime' property values"):
            if 'DateTimeOriginal' in cln_meta[file]:
                date = cln_meta[file]['DateTimeOriginal']
                try:
                    if date is not None:
                        tmp = date.replace('T', ' ').replace('-', ':').split('+')[0]
                        cln_meta[file]['DateTimeOriginal'] = datetime.datetime.strptime(tmp[:19], '%Y:%m:%d %H:%M:%S')
                        # if the year is after actual year, we assume that the date is wrong
                        if cln_meta[file]['DateTimeOriginal'].year > datetime.datetime.now().year:
                            date_error.append(cln_meta[file]['DateTimeOriginal'])
                            cln_meta[file]['DateTimeOriginal'] = None
                            cpt_error += 1
                        else:
                            cpt += 1
                except ValueError:
                    date_error.append(date)
                    cln_meta[file]['DateTimeOriginal'] = None
                    cpt_error += 1
    except Exception as e:
        print(e)

    print(f"Metadata cleaned ! {cpt}/{len(cln_meta)} dates OK, {cpt_error} dates KO")
    print(f"Dates KO : {date_error}")

    # Clean 'tags' property values
    for file in tqdm(cln_meta, desc="Clean 'tags' property values"):
        if 'tags' in cln_meta[file]:
            val = None
            if cln_meta[file]['tags'] is not None:
                val = eval(cln_meta[file]['tags'])
            cln_meta[file]['tags'] = val

    # Clean the GPS infos
    cln_meta = clean_gps_infos(cln_meta)

    return cln_meta

The function "get_metadata" is used to get the metadata from the database. If the metadata file already exists, it reads it. Otherwise, it gets the metadata from the database, cleans it, saves it in a CSV file and returns it.

In [ ]:
def get_metadata():
    """
    Get the metadata from the database
    :return: A JSON object with the metadata
    """
    # Check if the metadata file already exists
    if os.path.isfile('metadata.csv'):
        # If the file exists, read it
        return pd.read_csv('metadata.csv')
    else:
        # Get the metadata from the database
        # brut_metadata = get_metadata_from_mariadb_db(sql_database, sql_user, sql_password, sql_host)
        brut_metadata = get_metadata_from_mariadb_db(sql_database, sql_user, sql_password, sql_host)
        # Clean the metadata
        cln_metadata = clean_metadata(brut_metadata)
        # Convert the metadata to a DataFrame
        df_metadata = pd.DataFrame.from_dict(cln_metadata).transpose()
        # Fill the 'Make' property NaN values with 'Undefined'
        df_metadata['Make'].fillna('Undefined', inplace=True)

        df_metadata.to_csv('metadata.csv', index=False, mode='w')

        return df_metadata

In [ ]:
df_metadata = get_metadata()

### Overview of the metadata

The code below displays a little overview of the metadata.
The first give a general overview of the metadata.
The second displays the number data per property.

In [ ]:
df_metadata.head()

In [ ]:
print(f'Number of images : {len(df_metadata["filename"])}')
df_metadata.count()

# Define the functions to display the metadata

- "display_bar" is used to display a bar chart.
- "display_pie" is used to display a pie chart.
- "display_curve" is used to display a curve chart.
- "display_histogram" is used to display a histogram.
- "display_tree_map" is used to display a tree map.

In [ ]:
def display_bar(title, x_label, y_label, x_values, y_values, colors=None, rotation=90):
    """
    Display a bar chart

    :param title: The title of the chart
    :param x_label: The x-axis label
    :param y_label: The y-axis label
    :param x_values: The values of the x-axis
    :param y_values: The values of the y-axis
    :param colors: The colors of the bars
    :param rotation: The rotation of the x-axis labels
    """
    plt.bar(x_values, y_values, color=colors)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.xticks(rotation=rotation)
    plt.show()

In [ ]:
def display_pie(title, values, labels, colors=None, autopct="%1.1f%%", legend_title=None, legend_loc=None,
                legend_margin=None):
    """
    Display a pie chart

    :param title: The title of the chart
    :param values: The values of the chart
    :param labels: The labels of the chart
    :param colors: The colors of the chart
    :param autopct: The percentage format
    :param legend_title: The title of the legend,
    :param legend_loc: The location of the legend
    :param legend_margin: The margin of the legend
    """
    plt.pie(values, labels=labels, autopct=autopct, colors=colors)
    if legend_title is not None or legend_loc is not None or legend_margin is not None:
        plt.legend(title=legend_title, loc=legend_loc, bbox_to_anchor=legend_margin)
    plt.title(title)
    plt.show()

In [ ]:
def display_curve(title, x_label, y_label, x_values, y_values, rotation=90):
    """
    Display a curve

    :param title: The title of the curve
    :param x_label: The label of the x_axis
    :param y_label: The label of the y_axis
    :param x_values: The values of the x_axis
    :param y_values: The values of the y_axis
    :param rotation: The rotation of the x_axis labels
    """

    plt.plot(x_values, y_values)
    plt.xticks(rotation=rotation)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()

In [ ]:
def display_histogram(title, x_label, y_label, x_values, bins=10, rotation=90):
    """
    Display a histogram

    :param title: The title of the histogram
    :param x_label: The label of the x_axis
    :param y_label: The label of the y_axis
    :param x_values: The values of the x_axis
    :param bins: The number of bins
    :param rotation: The rotation of the x_axis labels
    """

    plt.hist(x_values, bins=bins)
    plt.xticks(rotation=rotation)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()

In [ ]:
def display_tree_map(title, sizes, labels, colors, alpha=0.6):
    """
    Display a tree map

    :param title: The title of the tree map
    :param sizes: The sizes of the tree map
    :param labels: The labels of the tree map
    :param colors: The colors of the tree map
    :param alpha: The alpha of the tree map
    """
    squarify.plot(sizes=sizes, label=labels, color=colors, alpha=alpha)
    plt.title(title)
    plt.show()

# Graph images : size (static)

The function "graph_images_size_static" is used to graph the number of images per size category. It takes a dataframe of the metadata as a parameter and return a graph of the number of images per size category. The interval size is 200 by default and the number of intervals is 4 by default.

In [ ]:
def graph_images_size_static(df_meta, interval_size=3000, nb_intervals=2):
    """
    Graph the number of images per size category

    :param df_meta: The metadata to graph
    :param interval_size: The size of the intervals
    :param nb_intervals: The number of intervals
    """

    # Calculate the minimum size of each image and store it in a new column
    df_meta['min_size'] = df_meta[['ImageWidth', 'ImageHeight']].min(axis=1)

    # Determine the maximum minimum size
    max_min_size = df_meta['min_size'].max()

    # Create a list of intervals based on the interval size and number of intervals
    inter = [i * interval_size for i in range(nb_intervals + 1)]

    # Create a list of labels for each interval
    labels = [f'{inter[i]}-{inter[i + 1]}' for i in range(nb_intervals)]

    # Categorize each image based on its size and interval
    df_meta['size_category'] = pd.cut(df_meta['min_size'], bins=inter, labels=labels)

    # Count the number of images in each category
    size_counts = df_meta['size_category'].value_counts()

    display_bar(title='Number of images per size category', x_label='Size category', y_label='Number of images',
                x_values=size_counts.index, y_values=size_counts.values)

In [ ]:
graph_images_size_static(df_metadata, 5000, 3)

# Graph images : size (dynamic)

The function "graph_images_size_dynamic" is used to graph the number of images per size category. It takes a dataframe of metadata as an argument and returns a graph with the number of images per size category. The interval size is calculated dynamically. The number of columns in the graph is 7 by default.

You can choose the type of graph to display (bar, pie or all for both).

In [ ]:
def graph_images_size_dynamic(df_meta, nb_intervals=7, graph_type='all'):
    """
    Graph the number of images per size category
    The interval size is calculated dynamically

    :param df_meta: The metadata to graph
    :param nb_intervals: The number of intervals in the graph
    :param graph_type: The type of graph to display (bar, pie or all for both)
    """

    # Calculate the minimum size of each image and store it in a new column
    df_meta['min_size'] = df_meta[['ImageHeight', 'ImageWidth']].min(axis=1)
    df_meta['min_size'].fillna(1, inplace=True)

    # Determine the maximum minimum size and calculate the number of bins dynamically based on the number of columns
    max_min_size = df_meta['min_size'].max()
    num_images = len(df_meta)
    num_bins = int(num_images / (num_images / nb_intervals))

    # Create a list of bins based on the maximum minimum size and number of bins
    bins = [i * (max_min_size / num_bins) for i in range(num_bins + 1)]

    # Create a list of labels for each bin
    labels = [f'{int(bins[i])}-{int(bins[i + 1])}' for i in range(num_bins)]

    # Categorize each image based on its size and bin
    df_meta['size_category'] = pd.cut(df_meta['min_size'], bins=bins, labels=labels)

    # Count the number of images in each category
    size_counts = df_meta['size_category'].value_counts()

    title = 'Number of images per size category'
    x_label = 'Image size'
    y_label = 'Number of images'

    # Create the appropriate chart based on the graph type parameter
    if graph_type == 'bar':
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=size_counts.index,
                    y_values=size_counts.values)
    elif graph_type == 'pie':
        display_pie(title=title, values=size_counts.values, labels=size_counts.index)
    elif graph_type == 'all':
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=size_counts.index,
                    y_values=size_counts.values)
        display_pie(title=title, values=size_counts.values, labels=size_counts.index)
    else:
        raise ValueError('Invalid graph type')


In [ ]:
graph_images_size_dynamic(df_metadata, 5, 'all')

# Graph images : DateTime

The function "graph_images_datetime" is used to graph the number of images per year. It takes a dataframe of metadata as an argument and returns a graph with the number of images per year.

You can choose the type of graph to display (bar, pie, curve or all for all).


In [ ]:
def graph_images_year(df_meta, nb_intervals=10, graph_type='all'):
    """
    Graph the number of images per year

    :param df_meta: The metadata to graph (expects a list of dictionaries)
    :param graph_type: The type of graph to display (bar, pie, curve or all for all)
    :param nb_intervals: The number of intervals to display
    """

    # Extract year from the 'DateTime' column and create a new 'Year' column
    df_meta['Year'] = pd.DatetimeIndex(df_meta['DateTimeOriginal']).year

    # Group the data by year and count the number of images for each year
    image_count = df_meta.groupby('Year').size().reset_index(name='count').sort_values('count', ascending=False)[
                  :nb_intervals]
    image_count['Year'] = image_count['Year'].astype(int)

    # Set the title of the graph
    title = 'Number of images per year'
    x_label = 'Year'
    y_label = 'Number of images'

    # Display different types of graphs based on the 'graph_type' parameter
    if graph_type == 'bar':
        # Display a bar chart
        image_count.plot(kind='bar', x='Year', y='count')
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=image_count['Year'],
                    y_values=image_count['count'])

    elif graph_type == 'pie':
        # Display a pie chart using a custom function 'display_pie'
        display_pie(title=title, values=image_count['count'], labels=image_count['Year'])

    elif graph_type == 'curve':
        # Display a line chart using a custom function 'display_curve'
        image_count = df_meta.groupby('Year').size().reset_index(name='count').sort_values('Year', ascending=True)
        display_curve(title=title, x_label=x_label, y_label=y_label, x_values=image_count['Year'],
                      y_values=image_count['count'])

    elif graph_type == 'all':
        # Display all three types of graphs: bar, pie, and line charts

        # Bar chart
        image_count.plot(kind='bar', x='Year', y='count')
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=image_count['Year'],
                    y_values=image_count['count'])

        # Pie chart
        display_pie(title=title, values=image_count['count'], labels=image_count['Year'])

        # Line chart
        image_count = image_count.sort_values('Year', ascending=True)
        display_curve(title=title, x_label=x_label, y_label=y_label, x_values=image_count['Year'],
                      y_values=image_count['count'])
    else:
        # Raise an error if an invalid 'graph_type' parameter is passed
        raise ValueError('Invalid graph type')

In [ ]:
graph_images_year(df_metadata, 10, 'all')

# Graph images : Brand

The function "graph_images_brand" is used to graph the number of images per brand. It takes a dataframe of metadata as an argument and returns a graph with the number of images per brand.

You can choose the type of graph to display (bar, pie or all for both), and the number of columns to display.

In [ ]:
def graph_images_brand(df_meta, graph_type='all', nb_columns=5):
    """
    Graph the number of images per brand

    :param df_meta: The metadata to graph
    :param graph_type: The type of graph to display (bar, pie or all for both)
    :param nb_columns: The number of columns to display
    """

    # Initialize an empty dictionary to store the counts of each brand
    counts = {}

    # Loop through each brand in the metadata and count the number of occurrences
    for make in df_meta['Make']:
        if make is not None:
            counts[make] = counts.get(make, 0) + 1

    sorted_counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))

    # Convert the dictionary into two lists of labels and values for graphing
    labels = list(sorted_counts.keys())[:nb_columns]
    values = list(sorted_counts.values())[:nb_columns]

    # Set the title for the graph
    title = 'Number of images per brand'
    x_label = 'Brand'
    y_label = 'Number of images'

    # Determine which type of graph to display based on the 'graph_type' parameter
    if graph_type == 'bar':
        # Display a bar graph
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=labels, y_values=values)
    elif graph_type == 'pie':
        # Display a pie chart
        display_pie(title=title, values=values, labels=labels)
    elif graph_type == 'all':
        # Display both a bar graph and a pie chart
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=labels, y_values=values)
        display_pie(title=title, values=values, labels=labels)
    else:
        # Raise an error if the 'graph_type' parameter is invalid
        raise ValueError('Invalid graph type')

In [ ]:
graph_images_brand(df_metadata, 'all', 10)

# Graph images : Images with GPS

#### Overview

The following code is used to give the count of images with GPS data.

In [ ]:
print(f"Number of images with GPS data : {df_metadata['Latitude'].count()}")

### Extract GPS coordinates

The function "get_coordinates" is used to extract the coordinates of the images with GPS data. It takes a dataframe of metadata as an argument and returns a dictionary with the coordinates of the images with GPS data.

In [ ]:
def get_coordinates(df_meta):
    """
    Extract the coordinates of the images with GPS data

    :param df_meta: The metadata to extract the coordinates from
    """
    coords = {}
    for file, lattitude, longitude, altitude in zip(
            df_meta['filename'],
            df_meta['Latitude'],
            df_meta['Longitude'],
            df_meta['Altitude']
    ):
        if lattitude is not None and not np.isnan(lattitude) \
                and longitude is not None and not np.isnan(longitude):
            coords.update({file: [lattitude, longitude, altitude]})

    return coords

In [ ]:
coordinates = get_coordinates(df_metadata)

### Using a Map with markers

The function "display_coordinates_on_map" is used to display the coordinates of the images with GPS data on a map. It takes a dictionary of coordinates as an argument and returns a map with the coordinates displayed as markers.

Inside the method comments, you can find a way to display the images as markers instead of the default markers.

In [ ]:
def display_coordinates_on_map(coordinates_list):
    """
    Display the coordinates on a map

    :param coordinates_list: The coordinates to display
    :return: The map with the coordinates displayed as markers
    """

    # create a map centered at a specific location
    m = folium.Map(location=[0, 0], zoom_start=1)

    # add markers for each set of coordinates
    for image, coords in coordinates_list.items():
        if coords != [0.0, 0.0, '0.000000']:
            lat, lon, alt = coords

            # Create a marker with the image as the icon
            # !warning! : the image must be download and you need to add :
            # from folium.features import CustomIcon

            # image_path = '../output/images/' + key
            # icon = CustomIcon(icon_image=image_path, icon_size=(100, 100))
            # folium.Marker(location=coord, icon=icon).add_to(m)
            folium.Marker(location=[lat, lon], tooltip=image, popup=f'file:{image}\ncoord:{coords}').add_to(m)
    return m

In [ ]:
display_coordinates_on_map(coordinates)

### Using graphs by country

#### get country

The get_country_single method is used to obtain the country corresponding to a single geographic coordinate. It takes a key and a coordinate as input, makes a request to the Nominatim service to obtain location information, extracts the country from the received data, and adds the country to the coordinate. In case of an error, it displays an error message with the concerned key and coordinate.

In [ ]:
def get_country_single(key, coord):
    """
    Get the country of a single coordinate

    :param key: The key for the coordinate
    :param coord: The coordinate to get the country from
    :return: The key and coordinate with the country added
    """
    # Create a geolocator
    geolocator = Nominatim(user_agent="geoapiExercises")

    # Check if the country hasn't been found yet
    if len(coord) < 4:
        try:
            # Get location information
            location = geolocator.reverse(coord, exactly_one=True, language='en')
            address = location.raw['address']
            # Get the country
            country = address.get('country')
            coord.append(country)
        except:
            print(f"Error with {key} : {coord}")

    return key, coord

The get_country method is used to obtain the country for each coordinate in a list of coordinates. It utilizes the get_country_single method to process each coordinate individually and optimizes the processing by using multithreading. The number of threads used can be specified using the num_threads parameter. The get_country method submits each coordinate to a ThreadPoolExecutor, which runs instances of get_country_single in parallel. The results are then gathered and returned as an updated coordinates dictionary with the countries added.

In [ ]:
def get_country(coordinates_list, num_threads=4):
    """
    Get the country of each coordinate using multithreading

    :param coordinates_list: The coordinates to get the country from
    :param num_threads: The number of threads to use for processing
    :return: The coordinates with the country added
    """
    # Create a dictionary to store the updated coordinates
    updated_coordinates = {}

    # Create a ThreadPoolExecutor with the specified number of threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit each coordinate to the executor for processing
        futures = [executor.submit(get_country_single, key, coord) for key, coord in coordinates_list.items()]

        # Iterate over the completed futures and store the results in the updated_coordinates dictionary
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(coordinates_list), desc='Getting country information'):
            try:
                key, coord = future.result()
                updated_coordinates[key] = coord
            except Exception as e:
                print(f"Error processing coordinates: {e}")

    # Return the updated coordinates dictionary with the countries added
    return updated_coordinates

In [ ]:
get_country(coordinates, 100)

### Display graphs

The function "graph_images_countries" is used to display graphs about the number of images by country. It takes a dictionary of coordinates as an argument and returns a graph.

The parameter "nb_inter" is used to set the number of interval to display. The parameter "graph" is used to set the type of graph to display (bar, pie, all).

In [ ]:
def graph_images_countries(coord_list, nb_inter=5, graph='all'):
    """
    Display graphs about the number of images by country

    :param coord_list: list of coordinates
    :param nb_inter: number of interval
    :param graph: type of graph to display (bar, pie, all)
    """

    # Create a pandas DataFrame from the coordinates dictionary
    df = pd.DataFrame.from_dict(coord_list, orient='index',
                                columns=['Latitude', 'Longitude', 'Altitude', 'Country'])

    # Group the DataFrame by continent and count the number of images
    country_count = df.groupby('Country')['Country'].count()
    country_count = country_count.sort_values(ascending=False)[:nb_inter]

    title = 'Number of images by country'
    x_label = 'Country'
    y_label = 'Image Count'

    if graph == 'bar':
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=country_count.index,
                    y_values=country_count.values)
    elif graph == 'pie':
        display_pie(title=title, values=country_count.values, labels=country_count.index)
    else:
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=country_count.index,
                    y_values=country_count.values)
        display_pie(title=title, values=country_count.values, labels=country_count.index)

In [ ]:
graph_images_countries(coordinates, 10, 'all')

### Using altitude

The function "graph_images_altitudes" is used to display graphs about the number of images by altitude. It takes a dictionary of coordinates as an argument and returns a graph.
You can use the parameter "nb_inter" to set the number of interval to display. The parameter "graph" is used to set the type of graph to display (histogram, pie, all).

In [ ]:
def graph_images_altitudes(coord_list, nb_inter=5, graph='all'):
    """
    Display graphs about the number of images by altitude.

    :param coord_list: list of coordinates
    :param nb_inter: number of interval
    :param graph: type of graph to display (histogram, pie, all)
    """
    # Extract altitudes from the given coordinates
    altitudes = []
    for img in coord_list:
        alt = float(coord_list[img][2])
        if alt > 0.0:
            altitudes.append(alt)

    # Create intervals using numpy's linspace()
    intervalles = np.linspace(0, max(altitudes), nb_inter + 1)

    # Convert the intervals into pairs of intervals
    intervalles = [(int(intervalles[i]), int(intervalles[i + 1])) for i in range(len(intervalles) - 1)]

    # Count how many altitudes are in each interval
    counts = [0] * len(intervalles)
    for altitude in altitudes:
        for i, intervalle in enumerate(intervalles):
            if intervalle[0] <= altitude < intervalle[1]:
                counts[i] += 1

    # Create a list of names for the intervals
    intervals_name = ["{}-{}".format(interval[0], interval[1]) for interval in intervalles]

    title = 'Number of images by altitude'
    x_label = 'Altitude'
    y_label = 'Image Count'

    # Display the chosen graph(s)
    if graph == 'histogram':
        display_histogram(title=title, x_label=x_label, y_label=y_label, x_values=altitudes, bins=nb_inter)
    elif graph == 'pie':
        display_pie(title=title, values=counts, labels=intervals_name)
    elif graph == 'bar':
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=intervals_name, y_values=counts)
    else:
        display_histogram(title=title, x_label=x_label, y_label=y_label, x_values=altitudes, bins=nb_inter)
        display_bar(title=title, x_label=x_label, y_label=y_label, x_values=intervals_name, y_values=counts)
        display_pie(title=title, values=counts, labels=intervals_name)

In [ ]:
graph_images_altitudes(coordinates, 5, 'all')

# Graph images : by Dominant Color

The function closest_colour is used to find the closest color in the webcolors library. It takes a color as an argument and returns the closest color.

In [ ]:
def closest_colour(requested_colour):
    """
    Find the closest color in the webcolors library

    :param requested_colour: color to find
    :return: the closest color
    """
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

The function get_colour_name is used to get the name of a color. It takes a color as an argument and returns the actual name and the closest name.

In [ ]:
def get_colour_name(requested_colour):
    """
    Get the name of the closest color

    :param requested_colour: color to find
    :return: the actual name and the closest name
    """
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name


The function "graph_images_dominant_color" is used to display graphs about the dominant color of images. It takes a dataframe of images as an argument and returns a graph.
You can use the parameter "nb_inter" to set the number of interval to display. The parameter "graph" is used to set the type of graph to display (bar, pie, treemap, all).

In [ ]:
def graph_dominant_colors(df_meta, nb_inter=5, graph='all'):
    """
    Display graphs about the number of images by dominant color

    :param df_meta: DataFrame of metadata
    :param nb_inter: number of interval
    :param graph: type of graph to display (bar, pie, treemap, all)
    """

    # Create a dictionary of dominant colors
    dict_dom_color = {}
    df_dict_meta = df_meta["dominant_color"].to_dict()

    # convert string of dom color to list
    for img in df_dict_meta:
        try:
            if df_dict_meta[img] is not None and df_dict_meta[img] is not np.nan:
                list_dom_color = eval(df_dict_meta[img])
                dict_dom_color.update({img: list_dom_color})
        except:
            print(f"Error with {img} : {df_dict_meta[img]}")

    # Count the number of times each color appears
    color_counts = Counter()
    for image_colors in dict_dom_color.values():
        for color, percentage in image_colors:
            color_counts[color] += percentage

    # Map hexadecimal codes to color names
    color_names = {}
    for code in color_counts.keys():
        try:
            rgb = webcolors.hex_to_rgb(code)
            actual, closest = get_colour_name(rgb)
            color_names[code] = closest
        except ValueError:
            pass

    # Create a dictionary of color percentages
    dict_res = {}
    for key, val in color_names.items():
        if val in dict_res:
            dict_res[val] += round(color_counts[key] / 100, 5)
        else:
            dict_res[val] = round(color_counts[key] / 100, 5)

    # Create a bar graph showing the dominant colors in the images
    if sum(dict_res.values()) > 100:
        raise Exception('Error : sum of percentages is greater than 100')

    columns = dict_res.__len__()
    if columns > nb_inter: columns = nb_inter

    # Sort the dictionary by value
    sorted_colors = sorted(dict_res.items(), key=lambda x: x[1], reverse=True)
    top_colors = dict(sorted_colors[:columns])
    color_labels = list(top_colors.keys())
    sizes = list(top_colors.values())
    color = [webcolors.name_to_hex(c) for c in top_colors]

    title = 'Top Colors'
    x_label = 'Color'
    y_label = 'Percentage'

    if graph == 'bar':
        display_bar(title=title, x_label=x_label, y_label=y_label, colors=top_colors.keys(),
                    x_values=top_colors.keys(), y_values=top_colors.values())
    elif graph == 'pie':
        display_pie(title=title, values=top_colors.values(), labels=top_colors.keys(), colors=color_labels)
    elif graph == 'treemap':
        display_tree_map(title=title, sizes=sizes, labels=color_labels, colors=color, alpha=.7)
    else:
        display_bar(title=title, x_label=x_label, y_label=y_label, colors=top_colors.keys(),
                    x_values=top_colors.keys(), y_values=top_colors.values())
        display_pie(title=title, values=top_colors.values(), labels=top_colors.keys(), colors=color_labels)
        display_tree_map(title=title, sizes=sizes, labels=color_labels, colors=color, alpha=.7)

In [ ]:
graph_dominant_colors(df_metadata, 20, 'all')

# Graph images : by Tags

The function "graph_top_tags" is used to display graphs about the most common tags on images. It takes a dataframe of images as an argument and returns a graph.
You can choose the number of tags to display and the type of graph to display (bar, pie, treemap, all).

In [ ]:
def graph_top_tags(df_meta, nb_inter=5, graph='all'):
    """
    Display graphs about the number of images by tags

    :param df_meta: DataFrame of metadata
    :param nb_inter: number of interval
    :param graph: type of graph to display (bar, pie, treemap, all)
    """

    all_tags = []
    for tags in df_meta['tags']:
        if tags is not None and tags is not np.nan:
            all_tags += tags

    top_tags = dict(collections.Counter(all_tags).most_common(nb_inter))

    title = 'Top Tags'
    x_label = 'Tag'
    y_label = 'Count'

    if graph == 'bar':
        display_bar(title=title, x_label=x_label, y_label=y_label,
                    x_values=top_tags.keys(), y_values=top_tags.values())
    elif graph == 'pie':
        display_pie(title=title, values=top_tags.values(), labels=top_tags.keys())
    else:
        display_bar(title=title, x_label=x_label, y_label=y_label,
                    x_values=top_tags.keys(), y_values=top_tags.values())
        display_pie(title=title, values=top_tags.values(), labels=top_tags.keys())

In [ ]:
graph_top_tags(df_metadata, 15, 'all')

## Tags by category

The function "categorize_tags" is used to categorize tags based on similarity to category prototypes. It takes a dataframe of images as an argument and returns a dictionary of categories.

In [ ]:
def categorize_tags(df_meta, categories_list : list):
    """
    Categorize tags based on similarity to category prototypes

    :param categories_list: list of categories
    :param df_meta: DataFrame of metadata
    :return: dictionary of categories
    """
    # Concatenate all tags in a list
    all_tags = []
    for tags in df_meta['tags']:
        try:
            tags = eval(tags)
            if tags is not None and tags is not np.nan:
                all_tags += tags
        except:
            print("Error : ", tags)

    try:
        nlp = spacy.load("en_core_web_md")  # load pre-trained word embedding model
    except OSError:
        !python -m spacy download en_core_web_lg

    categories = {}
    for cate in categories_list:
        categories[cate] = {}

    # categorize words based on similarity to category prototypes
    for word in tqdm(all_tags, desc="Categorizing tags"):
        # find the most similar category prototype for the word
        max_similarity = -1
        chosen_category = "other"
        for category in categories:
            similarity = nlp(word).similarity(nlp(category))
            if similarity > max_similarity:
                max_similarity = similarity
                chosen_category = category

        # add the word into the appropriate category dictionary
        categories[chosen_category].update({word: max_similarity})

    return categories

In [ ]:
cate_list= ['Fruit', 'Animal', 'Electronics', 'Furniture', 'Vehicle', 'Clothing', 'Sport', 'Kitchen', 'Outdoor', 'Accessory']
cat_tags = categorize_tags(df_metadata, cate_list)

## Graph tags by category

The function "graph_categorized_tags" is used to display a Denrogram of categorized tags. It takes a dictionary of categorized tags as an argument and returns a graph.

In [ ]:
def graph_categorized_tags(categorized_tags):
    """
    Display a Denrogram of categorized tags

    :param categorized_tags: dictionary of categorized tags
    """
    keys_and_subkeys = []
    for key, subdict in categorized_tags.items():
        for subkey in subdict:
            keys_and_subkeys.append((key, subkey))

    labels = [f"{key} -> {subkey}" for key, subkey in keys_and_subkeys]

    def custom_distance(x, y):
        key1, subkey1 = x
        key2, subkey2 = y
        if key1 == key2:
            return abs(categorized_tags[key1][subkey1] - categorized_tags[key2][subkey2])
        else:
            return 1.0

    dist_matrix = pdist(keys_and_subkeys, custom_distance)
    Z = linkage(dist_matrix, method='average')

    plt.figure(figsize=(10, 7))
    dendrogram(Z, labels=labels, orientation='top', leaf_font_size=10)
    plt.xlabel("Distance")
    plt.tight_layout()
    plt.show()

In [ ]:
graph_categorized_tags(cat_tags)